In [28]:
import pandas as pd
import numpy as np
import re
import nltk

nltk.download('stopwords')
nltk.download('wordnet')
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from Cleaner import Cleaner
from sklearn.pipeline import Pipeline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rahim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Rahim\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [29]:
df = pd.read_csv(r"../data/news_dataset.csv")

In [30]:
X, y = df.text, df.category

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [32]:
pipeline = Pipeline(steps=[
    ("Cleaner", Cleaner()),
    ("Tfidf", CountVectorizer(stop_words=stopwords.words('english'))),
    ("RandomForestClassifier", RandomForestClassifier(n_estimators=1000)),
])

In [33]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('Cleaner', Cleaner()),
                ('Tfidf',
                 CountVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('RandomForestClassifier',
                 RandomForestClassifier(n_estimators=1000))])

In [34]:
y_pred = pipeline.predict(X_test)

In [35]:
confusion_matrix(y_test, y_pred)

array([[107,   0,   2,   0,   1],
       [  3,  71,   1,   0,   1],
       [  3,   0,  68,   2,   0],
       [  0,   0,   1, 103,   0],
       [  2,   0,   1,   1,  78]], dtype=int64)

In [36]:
print(classification_report(y_test, y_pred))

               precision    recall  f1-score   support

     business       0.93      0.97      0.95       110
entertainment       1.00      0.93      0.97        76
     politics       0.93      0.93      0.93        73
        sport       0.97      0.99      0.98       104
         tech       0.97      0.95      0.96        82

     accuracy                           0.96       445
    macro avg       0.96      0.96      0.96       445
 weighted avg       0.96      0.96      0.96       445



In [37]:
accuracy_score(y_test, y_pred)

0.9595505617977528

In [38]:
with open(r'../models/CountVectorizerRandomForestClassifier', 'wb') as picklefile:
    pickle.dump(pipeline, picklefile)